In [1]:
from enum import Enum

In [2]:
from pathlib import Path
from glob import glob
from joblib import Parallel, delayed

In [3]:
import numpy as np
import pandas as pd
import datasets

In [21]:
from prettytable import PrettyTable

In [ ]:
from tqdm import tqdm

In [5]:
import sklearn

from sklearn import metrics
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, balanced_accuracy_score

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

In [6]:
base_path = './data/ecg_features_60s_clean_twa_rqa_60s'

In [7]:
X_labels =  [
    "hrv_mean",
    "hrv_min",
    "hrv_max",
    "hrv_std",
    "hrv_rms",
    "hr_mean",
    "hr_min",
    "hr_max",
    "rr_mean",
    "rr_min",
    "rr_max",
    "rr_std",
    "nn50",
    "pnn50",
    "rmssd",
    "lf",
    "hf",
    "vhf",
    "uhf",
    "tp",
    "lp_hf",
    "lp_vhf",
    "lp_uhf",
    "hf_normalized",
    "w",
    "wmax",
    "wen",
    "MeanNN",
    "SDNN",
    "SDANN1",
    "SDNNI1",
    "SDANN2",
    "SDNNI2",
    "SDANN5",
    "SDNNI5",
    "RMSSD",
    "SDSD",
    "CVNN",
    "CVSD",
    "MedianNN",
    "MadNN",
    "MCVNN",
    "IQRNN",
    "SDRMSSD",
    "Prc20NN",
    "Prc80NN",
    "pNN50",
    "pNN20",
    "MinNN",
    "MaxNN",
    "HTI",
    "TINN",
    "twa",
]
y_label = 'label'

In [8]:
participants = [Path(path).stem for path in glob(f'{base_path}/*.csv')]
train_participants, test_participants = train_test_split(participants, test_size=0.2, random_state=42)

In [9]:
def encode(baseline = 0, mental_stress = -1, high_physical_activity = -1, moderate_physical_activity = -1, low_physical_activity = -1):
    def inner(labels):
        baseline_list = ['Sitting', 'Recov1', 'Recov2', 'Recov3', 'Recov4', 'Recov5', 'Recov6']
        mental_stress_list = ['TA', 'SSST_Sing_countdown', 'Pasat', 'Raven', 'TA_repeat', 'Pasat_repeat']
        high_physical_stress_list = ['Treadmill1', 'Treadmill2', 'Treadmill3', 'Treadmill4', 'Walking_fast_pace', 'Cycling', 'stairs_up_and_down']
        moderate_physical_stress_list = ['Walking_own_pace', 'Dishes', 'Vacuum']
        low_physical_stress_list = ['Standing', 'Lying_supine', 'Recov_standing']
        
        def encode_multiclass(label):
            if label in baseline_list:
                return baseline
            elif label in mental_stress_list:
                return mental_stress
            elif label in high_physical_stress_list:
                return high_physical_activity
            elif label in moderate_physical_stress_list:
                return moderate_physical_activity
            elif label in low_physical_stress_list:
                return low_physical_activity
            else:
                return -1
            
        return {
            'label': [encode_multiclass(label) for label in labels],
        }
    return inner

def clean(dataset, mapping={}):
    dataset = dataset.map(
        encode(**mapping), 
        batched=True, 
        batch_size=2048, 
        input_columns=['label'],
        num_proc=4
    )
    return dataset.filter(
        lambda label: label != -1,
        input_columns=['label'],
    )

In [10]:
class Model(Enum):
    DecisionTree = 1
    RandomForest = 2
    AdaBoost = 3
    LinearDescriminantAnalysis = 4
    KNearestNeighbors = 5
    LogisticRegression = 6
    XGBoost = 7
    QuadraticDiscriminantAnalysis = 8

In [24]:
def train(model: Model, train_indices, val_indices, params= {}, mapping = {}):
    dataset = datasets.load_dataset(
        f'{base_path}', 
        train_participants=[train_participants[i] for i in train_indices],
        val_participants=[train_participants[i] for i in val_indices],
        test_participants=test_participants,
        trust_remote_code=True
    )

    dataset = clean(dataset, mapping=mapping)
    train = dataset['fit'].to_pandas().replace([np.inf, -np.inf, np.nan], 0)

    X_train, y_train = train[X_labels], train[y_label]
    del train

    if model == Model.DecisionTree:
        cls = DecisionTreeClassifier(criterion='entropy', min_samples_split=20, **params, random_state=42)
    elif model == Model.RandomForest:
        cls = RandomForestClassifier(**params, random_state=42, bootstrap=False)
    elif model == Model.AdaBoost:
        cls = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy', min_samples_split=20), n_estimators=100, **params)
    elif model == Model.LinearDescriminantAnalysis:
        cls = LinearDiscriminantAnalysis(**params)
    elif model == Model.KNearestNeighbors:
        cls = KNeighborsClassifier(n_neighbors=9, **params)
    elif model == Model.LogisticRegression:
        cls = LogisticRegression(**params)
    elif model == Model.XGBoost:
        cls = XGBClassifier(**params)
    elif model == Model.QuadraticDiscriminantAnalysis:
        cls = QuadraticDiscriminantAnalysis(**params)

    cls.fit(X_train, y_train)

    del X_train

    val = dataset['validate'].to_pandas().replace([np.inf, -np.inf, np.nan], 0)
    X_val, y_val = val[X_labels], val[y_label]
    del val

    test = dataset['test'].to_pandas().replace([np.inf, -np.inf, np.nan], 0)
    X_test, y_test = test[X_labels], test[y_label]
    del test

    data = {
        'val_accuracy': sklearn.metrics.accuracy_score(y_val, cls.predict(X_val)),
        'val_balanced_accuracy': sklearn.metrics.balanced_accuracy_score(y_val, cls.predict(X_val)),
        'test_accuracy': sklearn.metrics.accuracy_score(y_test, cls.predict(X_test)),
        'test_balanced_accuracy': sklearn.metrics.balanced_accuracy_score(y_test, cls.predict(X_test)),
    }

    if len(y_train.unique()) == 2:
        ## binary
        data['val_f1'] = sklearn.metrics.f1_score(y_val, cls.predict(X_val))
        data['test_f1'] = sklearn.metrics.f1_score(y_test, cls.predict(X_test))
    else: 
        # multiclass
        data['val_f1'] = sklearn.metrics.f1_score(y_val, cls.predict(X_val), average='micro')
        data['test_f1'] = sklearn.metrics.f1_score(y_test, cls.predict(X_test), average='micro')

    del y_train, X_val, y_val
    del X_test, y_test

    return data 

### Baseline & Mental Stress

In [33]:
def attempt(model: Model):
    table = PrettyTable()
    table.title = f'{model.name}'
    table.field_names = [
        '',
        'Validation F1', 
        'Test F1', 
        'Validation Accuracy', 
        'Test Accuracy', 
        'Validation Balanced Accuracy', 
        'Test Balanced Accuracy'
    ]

    with tqdm(total=3) as pbar:
        for mapping in [ { "mental_stress": 1 }, { "high_physical_activity": 1 }, { "baseline": -1, "mental_stress": 0, "high_physical_activity": 1 } ]:
            pbar.set_description(f'{model.name} - {mapping}')
            scores = Parallel(n_jobs=3)(delayed(train)(
                model,
                train_indices, 
                val_indices, 
                mapping=mapping
            ) for train_indices, val_indices in KFold(n_splits=5, shuffle=True, random_state=42).split(train_participants)) 
            df = pd.DataFrame(scores)

            titles = []
            if ("baseline" not in mapping) or (mapping["baseline"] != -1):
                titles.append("Baseline")
            if "mental_stress" in mapping and mapping["mental_stress"] >= 0:
                titles.append("Mental Stress")
            if "high_physical_activity" in mapping and mapping["high_physical_activity"] >= 0:
                titles.append("High Physical Activity")

            table.add_row([
                f"{' & '.join(titles)}",
                f"{round(df['val_f1'].mean() * 100, 2)}% ± {round(df['val_f1'].std() * 100, 2)}%", 
                f"{round(df['test_f1'].mean() * 100, 2)}% ± {round(df['test_f1'].std() * 100, 2)}%", 
                f"{round(df['val_accuracy'].mean() * 100, 2)}% ± {round(df['val_accuracy'].std() * 100, 2)}%", 
                f"{round(df['test_accuracy'].mean() * 100, 2)}% ± {round(df['test_accuracy'].std() * 100, 2)}%", 
                f"{round(df['val_balanced_accuracy'].mean() * 100, 2)}% ± {round(df['val_balanced_accuracy'].std() * 100, 2)}%", 
                f"{round(df['test_balanced_accuracy'].mean() * 100, 2)}% ± {round(df['test_balanced_accuracy'].std() * 100, 2)}%"
            ])
            pbar.update(1)
    
    print(table)

In [34]:
attempt(Model.DecisionTree)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                               DecisionTree                                                                              |
+----------------------------------------+----------------+----------------+---------------------+----------------+------------------------------+------------------------+
|                                        | Validation F1  |    Test F1     | Validation Accuracy | Test Accuracy  | Validation Balanced Accuracy | Test Balanced Accuracy |
+----------------------------------------+----------------+----------------+---------------------+----------------+------------------------------+------------------------+
|        Baseline & Mental Stress        | 58.51% ± 0.41% | 58.49% ± 0.24% |    55.42% ± 0.62%   | 55.51% ± 0.25% |        55.17% ± 0.77%   

In [39]:
for model in Model:
    if model == Model.DecisionTree:
        continue
    elif model == Model.RandomForest:
        continue
    attempt(model)

KeyboardInterrupt: 